# Unit 1 - Lesson 2 - Drill - PostgreSQL - Joins and CTEs

In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(host="localhost", database="BikeShare", user="user_1", password="****")
cur = conn.cursor()

### Query 1:  What are the three longest trips on rainy days?

In [3]:
q1 = '''
WITH
    rainy_days
AS (
    SELECT
        DATE(date) AS rain_date
    FROM
        weather
    WHERE
        events LIKE 'Rain'
    GROUP BY 1
)
SELECT
    t.trip_id,
    t.duration,
    DATE(t.start_date)
FROM
    trips t
JOIN
    rainy_days r
ON
    DATE(t.start_date) = r.rain_date
ORDER BY duration DESC
LIMIT 3;
'''

In [4]:
cur.execute(q1)
query1 = cur.fetchall()

In [5]:
query1[:5]

[('1173890', 85900, datetime.date(2016, 4, 22)),
 ('1009870', 84349, datetime.date(2015, 11, 15)),
 ('1210487', 83915, datetime.date(2016, 5, 21))]

### Query 2:  Which station is full most often?

In [6]:
q2 = '''
SELECT
   status.station_id,
   stations.name,
   COUNT(CASE WHEN status.docks_available=0 then 1 END) empty_count
FROM
    status
JOIN
    stations
ON
    stations.station_id = status.station_id
GROUP BY 1, 2
ORDER BY empty_count DESC;
'''

In [7]:
cur.execute(q2)
query2 = cur.fetchall()

In [8]:
query2[:5]

[(4, 'Santa Clara at Almaden', 5788),
 (3, 'San Jose Civic Center', 1444),
 (2, 'San Jose Diridon Caltrain Station', 626),
 (5, 'Adobe on Almaden', 0)]

### Query 3:  Return a list of stations with a count of number of trips starting at that station but ordered by dock count.

In [9]:
q3 = '''
SELECT
    trips.start_station,
    stations.dockcount,
    COUNT(*)
FROM
    trips
JOIN
    stations
ON
    stations.name = trips.start_station
GROUP BY 1, 2
ORDER BY 2 DESC
'''

In [10]:
cur.execute(q3)
query3 = cur.fetchall()

In [11]:
query3[:5]

[('Cyril Magnin St at Ellis St', 35, 69),
 ('5th St at Folsom St', 31, 173),
 ('2nd at Townsend', 27, 14099),
 ('Market at 10th', 27, 9937),
 ('Market at Sansome', 27, 10970)]

### Query 4:  (Challenge) What's the length of the longest trip for each day it rains anywhere?

In [12]:
q4 = '''
WITH
    rainy_days
AS (
    SELECT
        DATE(date) weather_date
    FROM
        weather
    WHERE
        Events LIKE 'Rain'
    GROUP BY 1
),
    rainy_trips
AS (
    SELECT
        trip_id,
        duration,
        DATE(trips.start_date) trip_date
    FROM
        trips
    JOIN
        rainy_days
    ON
        rainy_days.weather_date = DATE(trips.start_date)
    ORDER BY duration DESC
)
SELECT
    trip_date,
    MAX(duration) max_duration
FROM
    rainy_trips
GROUP BY 1
ORDER BY max_duration DESC;
'''

In [13]:
cur.execute(q4)
query4 = cur.fetchall()

In [14]:
query4[:5]

[(datetime.date(2016, 4, 22), 85900),
 (datetime.date(2015, 11, 15), 84349),
 (datetime.date(2016, 5, 21), 83915),
 (datetime.date(2016, 5, 7), 82896),
 (datetime.date(2016, 6, 17), 81687)]

In [15]:
cur.close()
conn.close()